In [ ]:
import json
from transformers import BertTokenizer
import torch

In [ ]:
json_name = f'/mnt/disk3/Alfred/Rescoring/data/aishell/dev/pll_data/token_pll_withLM.json'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
nbest = 50

In [ ]:
with open(json_name, 'r') as f, \
open(f'/mnt/disk3/Alfred/Rescoring/data/aishell/dev/pll_data/withLM/PLL_hyp.trn', 'w') as hyp, \
open(f'/mnt/disk3/Alfred/Rescoring/data/aishell/dev/pll_data/withLM/PLL_ref.trn', 'w') as ref:
    data = json.load(f)
    num = 0
    cers = []
    min_cer = 100
    best_weight = 0.0
    for num in range(100):
        weight = num * 0.01
        c = 0
        s = 0
        de = 0
        i = 0
        for n, d in enumerate(data):
            score = torch.tensor(d['score'][:nbest])
            pll = torch.tensor(d['pll'][:nbest])

            weight_sum = (1 - weight) * score + weight * pll
            
            mask_index = torch.argmax(weight_sum).item()


            c += d['err'][mask_index][0]
            s += d['err'][mask_index][1]
            de += d['err'][mask_index][2]
            i += d['err'][mask_index][3]
            ref_text = list(d['ref_text'])
            ref.write(f'{" ".join(ref_text)} (test_{n + 1})\n')
        cer = (s + de + i) / (c + s + de)
        cers.append(cer)
        if (cer < min_cer):
            min_cer = cer
            best_weight = weight
    print(best_weight)    
    print(min_cer)

In [ ]:
json_name = f'/mnt/disk3/Alfred/Rescoring/data/aishell/dev/pll_data/token_pll_noLM.json'

In [ ]:
json_name = f'/mnt/disk3/Alfred/Rescoring/data/aishell/test/pll_data/token_pll_withLM.json'

In [ ]:
with open(json_name, 'r') as f, \
    open(f'/mnt/disk3/Alfred/Rescoring/data/aishell/dev/pll_data/noLM/PLL_hyp.trn', 'w') as hyp, \
    open(f'/mnt/disk3/Alfred/Rescoring/data/aishell/dev/pll_data/noLM/PLL_ref.trn', 'w') as ref:
    data = json.load(f)
    weight = best_weight
    print(weight)
    c = 0
    s = 0
    de = 0
    i = 0
    for n, d in enumerate(data):
        score = torch.tensor(d['score'][:nbest])
        pll = torch.tensor(d['pll'][:nbest])

        weight_sum = (1-weight) * score + weight * pll
            
        mask_index = torch.argmax(weight_sum).item()
        max_text = list(d['text'][mask_index])


        hyp.write(f'{" ".join(max_text)} (test_{n + 1})\n')

        c += d['err'][mask_index][0]
        s += d['err'][mask_index][1]
        de += d['err'][mask_index][2]
        i += d['err'][mask_index][3]
        ref_text = list(d['ref_text'])
        ref.write(f'{" ".join(ref_text)} (test_{n + 1})\n')
    cer = (s + de + i) / (c + s + de)
    print(cer)

